In [2]:
import keras.backend as K
from keras import Input, Model, Sequential
from keras.layers import *
from keras.optimizers import Adam, RMSprop
from keras.callbacks import *
from pathlib import Path

/home/luca/.local/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [6]:
normalized_transactions_filepath = Path('../../datasets/berka_dataset/usable/normalized_transactions_100.npy')
generated_transactions_filepath = Path('../generative_models/wgan/2018-04-22_18-38-39/generated_datasets/last.npy')

timesteps = 100
elements = 50000

real_data = np.load(normalized_transactions_filepath)[:elements]
fake_data = np.load(generated_transactions_filepath)[:elements]

real_data.shape, fake_data.shape

((50000, 100), (50000, 100))

In [7]:
split = 0.2

x_train = np.vstack([real_data, fake_data]) 
y_train = np.vstack([np.zeros((elements, 1)), np.ones((elements, 1))])

shuffled_indexes = np.random.permutation(x_train.shape[0])

x_train = x_train[shuffled_indexes]
y_train = y_train[shuffled_indexes]

split_index = int(2*elements*split)

x_test, x_train = x_train[:split_index], x_train[split_index:]
y_test, y_train = y_train[:split_index], y_train[split_index:]


x_train.shape, y_train.shape, x_test.shape, y_test.shape

((80000, 100), (80000, 1), (20000, 100), (20000, 1))

In [9]:
model_inputs = Input((timesteps,))
classified = model_inputs

classified = Lambda(lambda x: K.expand_dims(x))(classified)
while classified.shape[1] > 1:
    classified = Conv1D(32, 3, activation='relu', padding='same')(classified)
    classified = MaxPooling1D(2, padding='same')(classified)
classified = Flatten()(classified)
classified = Dense(15, activation='relu')(classified)
classified = Dense(5, activation='relu')(classified)
classified = Dense(1, activation = 'sigmoid')(classified) 

classifier = Model(model_inputs, classified, 'classifier')
classifier.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [10]:
classifier.fit(x=x_train, y=y_train, validation_data=(x_test, y_test), epochs=5)

Train on 80000 samples, validate on 20000 samples
Epoch 1/5
80000/80000 [==============================] - 41s 517us/step - loss: 0.0120 - acc: 0.9941 - val_loss: 1.3547e-05 - val_acc: 1.0000
Epoch 2/5
52864/80000 [==================>...........] - ETA: 13s - loss: 3.2778e-05 - acc: 1.0000

KeyboardInterrupt: 